In [ ]:
!pip install PyPDF2 Flask transformers torch chromadb streamlit


In [ ]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

# Extract text from your PDF
pdf_text = extract_text_from_pdf('/content/gemma-model.pdf')


In [ ]:
pdf_text

"3/10/24, 8:32 PM Gemma: Google introduces new state-of-the-art open models\nhttps://blog.google/technology/developers/gemma-open-models/ 1/5DEVELOPERS\nGemma: Introducing new state-\nof-the-art open models\nGemma is built for responsible AI development from the same research and\ntechnology used to create Gemini models.\nFeb 21, 2024·3 min read\nJJeanine Banks\nVP & GM, Developer X and\nDevRelTTris Warkentin\nDirector, Google DeepMindShare\nListen to article\n7 minutes\nThe Keyword3/10/24, 8:32 PM Gemma: Google introduces new state-of-the-art open models\nhttps://blog.google/technology/developers/gemma-open-models/ 2/5At Google, we believe in making AI helpful for everyone. We have a long history of contributing innovations\nto the open community, such as with Transformers, TensorFlow, BERT, T5, JAX, AlphaFold, and AlphaCode.\nToday, we’re excited to introduce a new generation of open models from Google to assist developers and\nresearchers in building AI responsibly.\nGemm a open mod

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Split the text into paragraphs (or other smaller chunks)
paragraphs = pdf_text.split('\n\n')

# Vectorize the paragraphs
vectorizer = TfidfVectorizer()
paragraph_vectors = vectorizer.fit_transform(paragraphs)


In [ ]:
paragraph_vectors

<1x432 sparse matrix of type '<class 'numpy.float64'>'
	with 432 stored elements in Compressed Sparse Row format>

In [ ]:
from flask import Flask, request, jsonify
from transformers import pipeline
import numpy as np

app = Flask(__name__)

# Load the vectorized data and the vectorizer
doc_vectors = paragraph_vectors
vectorizer = TfidfVectorizer()
# Initialize QA model with a more powerful model
qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")


config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def retrieve_document(query):
    query_vector = vectorizer.transform([query])
    similarities = np.dot(doc_vectors, query_vector.T).toarray().flatten()
    best_match_index = similarities.argmax()
    return paragraphs[best_match_index]

In [ ]:
# prompt: save the model

import pickle

# Save the vectorizer
with open('vectorizer.pkl', 'wb') as f:
  pickle.dump(vectorizer, f)

# Save the document vectors
with open('doc_vectors.pkl', 'wb') as f:
  pickle.dump(doc_vectors, f)


In [ ]:
# prompt: streamlit webb app

%%writefile app.py
import streamlit as st
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import pipeline
import numpy as np

# Load the vectorizer and document vectors
with open('vectorizer.pkl', 'rb') as f:
  vectorizer = pickle.load(f)
with open('doc_vectors.pkl', 'rb') as f:
  doc_vectors = pickle.load(f)

# Load the QA pipeline
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

# Function to retrieve relevant document
def retrieve_document(query, doc_vectors, vectorizer):
    query_vector = vectorizer.transform([query])
    similarities = np.dot(doc_vectors, query_vector.T).toarray().flatten()
    best_match_index = similarities.argmax()
    return paragraphs[best_match_index]  # Assuming 'paragraphs' is available globally

# Streamlit app
st.title("PDF Question Answering App")

# User input for question
question = st.text_input("Enter your question:")

# Answer generation
if st.button("Get Answer"):
    if question:
        relevant_doc = retrieve_document(question, doc_vectors, vectorizer)
        result = qa_pipeline(question=question, context=relevant_doc)
        answer = result['answer']
        st.write("Answer:", answer)
    else:
        st.warning("Please enter a question.")

!streamlit run app.py


Writing app.py


In [ ]:
# prompt: paragraphs is not define

import PyPDF2
from sklearn.feature_extraction.text import TfidfVectorizer
from flask import Flask, request, jsonify
from transformers import pipeline
import numpy as np
import pickle
import streamlit as st
!pip install PyPDF2 Flask transformers torch chromadb streamlit


def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

# Extract text from your PDF
pdf_text = extract_text_from_pdf('/content/gemma-model.pdf')

# Split the text into paragraphs (or other smaller chunks)
paragraphs = pdf_text.split('\n\n') # Define paragraphs here

# Vectorize the paragraphs
vectorizer = TfidfVectorizer()
paragraph_vectors = vectorizer.fit_transform(paragraphs)

app = Flask(__name__)

# Load the vectorized data and the vectorizer
doc_vectors = paragraph_vectors
vectorizer = TfidfVectorizer()
qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")
def retrieve_document(query):
    query_vector = vectorizer.transform([query])
    similarities = np.dot(doc_vectors, query_vector.T).toarray().flatten()
    best_match_index = similarities.argmax()
    return paragraphs[best_match_index]
@app.route('/answer', methods=['POST'])
def answer():
    data = request.json
    question = data.get("question")
    relevant_doc = retrieve_document(question)
    result = qa_pipeline(question=question, context=relevant_doc)
    answer = result['answer']
    return jsonify({"answer": answer})

if __name__ == '__main__':
    app.run(debug=True)


# Save the vectorizer
with open('vectorizer.pkl', 'wb') as f:
  pickle.dump(vectorizer, f)

# Save the document vectors
with open('doc_vectors.pkl', 'wb') as f:
  pickle.dump(doc_vectors, f)


%%writefile app.py
import PyPDF2 # Import PyPDF2 here
from sklearn.feature_extraction.text import TfidfVectorizer
from flask import Flask, request, jsonify
from transformers import pipeline
import numpy as np
import pickle
import streamlit as st

# Load the vectorizer and document vectors
with open('vectorizer.pkl', 'rb') as f:
  vectorizer = pickle.load(f)
with open('doc_vectors.pkl', 'rb') as f:
  doc_vectors = pickle.load(f)

# Load the QA pipeline
qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")

# Extract text from your PDF (Assuming the PDF is still available)
pdf_text = extract_text_from_pdf('/content/gemma-model.pdf')
paragraphs = pdf_text.split('\n\n') # Define paragraphs here as well

# Function to retrieve relevant document
def retrieve_document(query, doc_vectors, vectorizer):
    query_vector = vectorizer.transform([query])
    similarities = np.dot(doc_vectors, query_vector.T).toarray().flatten()
    best_match_index = similarities.argmax()
    return paragraphs[best_match_index]

# Streamlit app
st.title("PDF Question Answering App")

# User input for question
question = st.text_input("Enter your question:")

# Answer generation
if st.button("Get Answer"):
    if question:
        relevant_doc = retrieve_document(question, doc_vectors, vectorizer)
        result = qa_pipeline(question=question, context=relevant_doc)
        answer = result['answer']
        st.write("Answer:", answer)
    else:
        st.warning("Please enter a question.")

!streamlit run app.py


In [ ]:
import pandas as pd
import random

# Load positive and negative texts from files
# Specify the encoding as 'latin-1' to handle the problematic characters
with open('positive.txt', 'r', encoding='latin-1') as file:
    positive_texts = file.readlines()

with open('negative.txt', 'r', encoding='latin-1') as file:
    negative_texts = file.readlines()

In [ ]:
positive_labels = ['Positive'] * len(positive_texts)
negative_labels = ['Negative'] * len(negative_texts)

In [ ]:
platforms = ['Twitter', 'Facebook', 'Instagram', 'LinkedIn', 'Reddit']

In [ ]:
positive_platforms = [random.choice(platforms) for _ in range(len(positive_texts))]
negative_platforms = [random.choice(platforms) for _ in range(len(negative_texts))]

In [ ]:
data = {
    'Text': positive_texts + negative_texts,
    'Sentiment': positive_labels + negative_labels,
    'Platform': positive_platforms + negative_platforms
}

df = pd.DataFrame(data)

In [ ]:
df.head()

Text Sentiment   Platform
0  the rock is destined to be the 21st century's ...  Positive  Instagram
1  the gorgeously elaborate continuation of " the...  Positive   LinkedIn
2                   effective but too-tepid biopic\n  Positive  Instagram
3  if you sometimes like to go to the movies to h...  Positive    Twitter
4  emerges as something rare , an issue movie tha...  Positive   Facebook

In [ ]:
df.to_csv('CyberBulling Dataset.csv', index=False)

In [ ]:
import pandas as pd
import glob

# Get all CSV files in the directory
csv_files = glob.glob('*.csv')

# Load each CSV file and store them in a list
dataframes = [pd.read_csv(file) for file in csv_files]

# Concatenate all DataFrames into one
combined_df = pd.concat(dataframes, ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv('combined_dataset.csv', index=False)


In [ ]:
data=pd.read_csv('combined_dataset.csv')

<ipython-input-10-eb238c266d0e>:1: DtypeWarning: Columns (1,2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('combined_dataset.csv')


In [ ]:
def label_cyberbullying(text):
    if any(word in text.lower() for word in ['hate', 'disgust', 'bigot']):
        return 'Hate Speech'
    elif any(word in text.lower() for word in ['anger', 'rage', 'fury']):
        return 'Aggression'
    elif any(word in text.lower() for word in ['stupid', 'idiot', 'dumb']):
        return 'Insults'
    elif any(word in text.lower() for word in ['toxic', 'vile', 'filthy']):
        return 'Toxicity'
    else:
        return 'None'


In [ ]:
# Ensure the 'Text' column is of string type
data['Text'] = data['Text'].astype(str)

# Apply the 'label_cyberbullying' function
data['Cyber-Bullying'] = data['Text'].apply(label_cyberbullying)

In [ ]:
platforms = ['Twitter', 'Facebook', 'Instagram', 'LinkedIn', 'Reddit']

# Randomly assign platforms
data['Platform'] = [random.choice(platforms) for _ in range(len(data))]

In [ ]:
data.to_csv('cyberbullying_data.csv', index=False)